<a href="https://colab.research.google.com/github/gvanathip/WebScrap_to_Visualization/blob/main/PropertyPrice_Webscrap_BarChartRaceAnimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

In [ ]:
import pandas as pd

In [ ]:
import time

In [ ]:
driver.get("https://www.numbeo.com/property-investment/rankings_by_country.jsp")

In [ ]:
p = driver.find_element("xpath",'//*[@id="page_container"]/div[2]/div[1]/form[1]/select').get_attribute("innerText")
time.sleep(5)
p

In [ ]:
y = []
for i in p.split("\n"):
  if i.isdigit():
    print(i)
    y.append(i)

In [ ]:
df = pd.DataFrame()
for i in y:
  print("https://www.numbeo.com/property-investment/rankings_by_country.jsp?title="+i)
  driver.get("https://www.numbeo.com/property-investment/rankings_by_country.jsp?title="+i)
  data = driver.find_element("xpath",'//*[@id="t2"]').get_attribute("innerText")
  time.sleep(5)
  col = data.split("\n\n")[0].split("\n\t\n")
  value = []
  for j in data.split("\n\n")[1].split("\n"):
    # print(j.split("\t"))
    value.append(j.split("\t"))
  df_new = pd.DataFrame(columns=col, data=value)
  df_new["year"] = i
  df = df.append(df_new)
  print(df)

In [ ]:
df

In [ ]:
Property_Price_to_Income = df[["Country","year","Price To Income Ratio"]]

In [ ]:
Property_Price_to_Income.year = pd.to_datetime(Property_Price_to_Income.year, format='%Y')

In [ ]:
Property_Price_to_Income

In [ ]:
Property_Price_to_Income = Property_Price_to_Income.pivot(index='year', columns='Country', values='Price To Income Ratio')

In [ ]:
Property_Price_to_Income

In [ ]:
pip install bar_chart_race

In [ ]:
Property_Price_to_Income = Property_Price_to_Income.fillna(0)

In [ ]:
for col in Property_Price_to_Income:
    Property_Price_to_Income[col] = pd.to_numeric(Property_Price_to_Income[col], errors='coerce')

In [ ]:
Property_Price_to_Income

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from matplotlib.animation import FuncAnimation

In [ ]:
import bar_chart_race as bcr

In [ ]:
plt.rcParams.update({# Use mathtext, not LaTeX
                            'text.usetex': False,
                            # Use the Computer modern font
                            'font.family': 'serif',
                            'font.serif': 'cmr10',
                            'mathtext.fontset': 'cm',
                            # Use ASCII minus
                            'axes.unicode_minus': False,
                            })

In [ ]:
df2 = Property_Price_to_Income.reset_index()
df2.index = df2.index * 12
last_idx = df2.index[-1] + 1
df_expanded = df2.reindex(range(last_idx))

In [ ]:
df_expanded['year'] = df_expanded['year'].fillna(method='ffill')
df_expanded = df_expanded.set_index('year')
df_expanded = df_expanded.interpolate()

In [ ]:
df_expanded 

In [ ]:
bcr.bar_chart_race(df = df_expanded, 
filename = None, 
filter_column_colors = True, 
cmap = "prism", 
title = "Property Price to Income",
n_bars=20,
steps_per_period=5,
interpolate_period=False,
sort="desc",
shared_fontdict={'family' : 'Helvetica', 'color' : '.1'},
scale='linear',)